## Important parameters
<br>xt_ocean: longitude, length 3600
<br>yt_ocean: latitude, length 1500
<br> [mind map](https://miro.com/app/board/o9J_lM4N1Pg=/?fromRedirect=1)

In [1]:
import netCDF4 as nc4
import keras
import xarray,numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, add
from keras.layers.core import  Dense, Dropout, Activation, Flatten, Reshape
from keras import regularizers
from keras.regularizers import l2
from keras.layers.convolutional import Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D
from keras.utils import np_utils
from tensorflow.keras import layers

# Read data

In [ ]:
file_path =['C:/Users/myxll/OneDrive - The University of Melbourne/capstone/data/OFAM_2017/ocean_eta_t/ocean_eta_t_2000_01.nc']
ds = nc4.MFDataset(file_path)
eta_t = ds.variables['eta_t'][:]
eta_t_numpy = np.array(eta_t)
eta_t_numpy = np.nan_to_num(eta_t_numpy)
eta_t_numpy.shape

In [ ]:
input_data = eta_t_numpy[0,:,:]
input_data.shape

In [ ]:
input_data[0]

# fixed missing values with 0

In [ ]:
for i in range(0,len(input_data)):
    arr = input_data[i]
    arr[arr == -32768] = 0

In [ ]:
input_data[0]

# Normalize the input

In [ ]:
def normalize_matrix(matrix):
    norm = np.linalg.norm(matrix)
    print(norm)
    matrix = matrix/norm  # normalized matrix
    return matrix

normalize_matrix(input_data)
input_data

In [ ]:
input_data[-40]

# Split array into 128*128

In [ ]:
import math
lat = 128
long = 128

pad_rows = math.ceil(3600/lat)*lat - 3600
pad_cols = math.ceil(1500/long)*long - 1500
print(pad_rows)
print(pad_cols)

In [ ]:
input_data=np.pad(input_data,((0,pad_cols),(0,pad_rows)), 'constant',constant_values=(0,0))

In [ ]:
input_data.shape

In [ ]:
l = np.array_split(input_data,len(input_data)/lat,axis=0)
input_data_split = []
for i in range(len(l)):
    dd = np.array_split(l[i],29,axis=1)
    input_data_split += dd
input_data_split = np.array(input_data_split)

In [ ]:
input_data_split.shape

# split train set & validation set

In [ ]:
train_set, test_set = train_test_split(input_data_split, test_size=0.33)
train_set.shape

# Autoencoder

In [ ]:
input_img = keras.Input(shape=(128, 128,1))

x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu',strides=(1,1))(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(4, (3, 3), activation='relu', padding='same',strides=(2,2))(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

x = layers.Conv2DTranspose(8, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2DTranspose(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2DTranspose(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((4, 4))(x)
decoded = layers.Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()

In [ ]:
history = autoencoder.fit(train_set, train_set,
                epochs=100, validation_data=(test_set, test_set))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
history.history